In [ ]:
!pip install accelerate bitsandbytes
!pip install flash-attn
!pip install git+https://github.com/huggingface/transformers.git
!pip install evaluate rouge_score

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", torch_dtype=torch.bfloat16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
model_size = sum(p.numel() for p in model.parameters())
print(f"Model Params: {model_size} parameters")

model_size_gb = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024 ** 3)
print(f"Model size in RAM: {model_size_gb:.4f} GB")

Model Params: 2506172416 parameters
Model size in RAM: 4.6681 GB


In [ ]:
#CNN/DM Dataset
df=pd.read_csv('/content/cnn_dm_dataset_200.csv')

In [ ]:
articles = df[0:50]['article']
dataset_summaries = df[0:50]['highlights']

docs=[]
model_summaries = []

for idx, article in enumerate(articles):
    chat = [
        { "role": "user", "content": "Provide a Concise Summary of: " +  article },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    print(prompt)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

    dataset_text_output = dataset_summaries[idx]

    generation_config = GenerationConfig(max_new_tokens=512)

    generated_ids = model.generate(input_ids=inputs.to(model.device),  generation_config=generation_config)

    output_p=tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    end_token_index = output_p.find('<eos>')
    if end_token_index != -1:
      output_p = output_p[:end_token_index].strip()
    else:
      last_period_index = output_p.rfind('.')
      output_p = output_p[:last_period_index + 1].strip()

    split_text_p = output_p.split("<start_of_turn>model")
    model_text_output = split_text_p[1].strip() if len(split_text_p) > 1 else print("Model Output: No Output")

    docs.append(article)
    model_summaries.append(model_text_output)

zipped_summaries = list(zip(docs, dataset_summaries, model_summaries))

df_res = pd.DataFrame(zipped_summaries, columns = ['article', 'dataset_summaries', 'model_summaries'])

df_res.to_csv("sum_cnndm_gemma2b-it_0shot.csv", index=False)

In [ ]:
df_res=pd.read_csv("/content/sum_cnndm_gemma2b-it_0shot.csv")
df_res

,article,dataset_summaries,model_summaries
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,"Sure, here's a concise summary of the article:..."
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...","Sure, here's a concise summary:\n\nA stray dog..."
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,"Sure, here's a concise summary of the passage:..."
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,"Sure, here's a concise summary:\n\nFive Americ..."
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,"Sure, here's a concise summary:\n\nA Duke stud..."
5,(CNN)He's a blue chip college basketball recru...,College-bound basketball star asks girl with D...,"Sure, here's a concise summary:\n\nTrey Moses,..."
6,(CNN)Governments around the world are using th...,Amnesty's annual death penalty report catalogs...,"Sure, here's a concise summary of the article:..."
7,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,"Sure, here's a concise summary:\n\nAndrew Gett..."
8,(CNN)Filipinos are being warned to be on guard...,"Once a super typhoon, Maysak is now a tropical...","Sure, here's a concise summary:\n\n- Tropical ..."
9,"(CNN)For the first time in eight years, a TV l...","Bob Barker returned to host ""The Price Is Righ...","Sure, here's a concise summary:\n\nA TV legend..."


In [ ]:
import evaluate
rouge = evaluate.load('rouge')

human_baseline_summaries = df_res['dataset_summaries'].values

original_model_results = rouge.compute(
    predictions=df_res['model_summaries'],
    references=human_baseline_summaries[0:len(df_res['model_summaries'])],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2987567725969439, 'rouge2': 0.11476334239907528, 'rougeL': 0.20710121822881514, 'rougeLsum': 0.25003333910678427}


#PHI-2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", device_map="auto", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model_size = sum(p.numel() for p in model.parameters())
print(f"Model Params: {model_size} parameters")

model_size_gb = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024 ** 3)
print(f"Model size in RAM: {model_size_gb:.4f} GB")

Model Params: 2779683840 parameters
Model size in RAM: 10.3551 GB


In [ ]:
articles = df[0:50]['article']
dataset_summaries = df[0:50]['highlights']

docs=[]
model_summaries = []

for idx, article in enumerate(articles):
  prompt = f"""
            Instruct: Generate an abstractive summary that highlights the main concepts within the document: {article} '\n'
            Output:
            """

  inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to('cuda')

  dataset_text_output = dataset_summaries[idx]

  generated_ids = model.generate(**inputs, max_new_tokens=256, pad_token_id= tokenizer.unk_token_id, forced_eos_token_id = tokenizer.eos_token_id)

  output_p=tokenizer.batch_decode(generated_ids)[0]
  end_token_index = output_p.find(".<|endoftext|>")
  if end_token_index != -1:
    output_p = output_p[:end_token_index].strip()
  else:
    last_period_index = output_p.rfind('.')
    output_p = output_p[:last_period_index + 1].strip()

  split_text_p = output_p.split("Output:")
  output = split_text_p[1].strip() if len(split_text_p) > 1 else print("Model Output: No Output")

  docs.append(article)
  model_summaries.append(output)

zipped_summaries = list(zip(docs, dataset_summaries, model_summaries))

df_res = pd.DataFrame(zipped_summaries, columns = ['article', 'dataset_summaries', 'model_summaries'])

df_res.to_csv("sum_cnndm_phi2_0shot.csv", index=False)

Model Output: No Output
Model Output: No Output


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [ ]:
df_res=pd.read_csv("/content/sum_cnndm_phi2_0shot.csv")
df_res

,article,dataset_summaries,model_summaries
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,Instruct: Generate an abstractive summary that...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",Summary: A stray dog in Washington State survi...
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,Instruct: Generate an abstractive summary that...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,The five Americans who were monitored for thre...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,The Duke student has admitted to hanging a noo...
5,(CNN)He's a blue chip college basketball recru...,College-bound basketball star asks girl with D...,The article is about a high school basketball ...
6,(CNN)Governments around the world are using th...,Amnesty's annual death penalty report catalogs...,Instruct: Generate an abstractive summary that...
7,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,"Andrew Getty, one of the heirs to billions of ..."
8,(CNN)Filipinos are being warned to be on guard...,"Once a super typhoon, Maysak is now a tropical...",The Philippine government has issued a warning...
9,"(CNN)For the first time in eight years, a TV l...","Bob Barker returned to host ""The Price Is Righ...","The TV legend Bob Barker, who hosted ""The Pric..."


In [ ]:
import evaluate
rouge = evaluate.load('rouge')

human_baseline_summaries = df_res['dataset_summaries'].values

original_model_results = rouge.compute(
    predictions=df_res['model_summaries'],
    references=human_baseline_summaries[0:len(df_res['model_summaries'])],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2118341165073755, 'rouge2': 0.09361548203992222, 'rougeL': 0.15381386287603235, 'rougeLsum': 0.17384491793747717}
